In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *  # noqa: F403

In [ ]:
#| export
import typer
from rich.console import Console
from tui_writer.tui import TranscriptionTUI

# Console for rich formatting
console = Console()



## CLI - Command Line Interface

This module provides both a TUI (Terminal User Interface) and CLI commands for audio transcription using Whisper models.

**Features:**
- Interactive TUI with real-time recording
- Model selection dialog
- Keyboard shortcuts for efficient workflow
- Clipboard integration



## Usage

### Launch the TUI (Default)

```bash
tui_writer
```


### Explicit TUI Launch

```bash
python -m tui_writer.cli tui
```

## Commands


In [ ]:

#| export
app = typer.Typer()

@app.command()
def tui():
    """Launch the interactive TUI for audio transcription."""
    tui_app = TranscriptionTUI()
    tui_app.run()
